<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning

This notebook should be run using with **Default Spark 3.3 & Python 3.10** or **Python 3.9** runtime environment. **If you are viewing this in Watson Studio and do not see Python 3.10.x in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following services:
  * Watson OpenScale
  * Watson Machine Learning 
  * DB2

  
The notebook will train, create and deploy a German Credit Risk model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

### Contents

- [Setup](#setup)
- [Model building and deployment](#model)
- [OpenScale configuration](#openscale)
- [Quality monitor and feedback logging](#quality)
- [Fairness, drift monitoring and explanations](#fairness)
- [Custom monitors and metrics](#custom)
- [Historical data](#historical)

# Setup <a name="setup"></a>

## Package installation

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install --upgrade pyspark==3.3.0 --no-cache | tail -n 1

#If you are running this notebook in non IBM Watson Studio env then uncomment the below pip statements and run it
#!pip install --upgrade pandas==1.2.3 --no-cache | tail -n 1 
#!pip install --upgrade requests==2.23 --no-cache | tail -n 1
#!pip install numpy==1.20.1 --no-cache | tail -n 1
#!pip install SciPy --no-cache | tail -n 1
#!pip install lime --no-cache | tail -n 1

!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

### Action: restart the kernel!

## Configure credentials

- WOS_CREDENTIALS (CP4D)
- WML_CREDENTIALS (CP4D)
- DATABASE_CREDENTIALS (DB2 on CP4D or Cloud Object Storage (COS))
- SCHEMA_NAME

In [3]:
WOS_CREDENTIALS = {
    "url": "<URL>",
    "username": "<USER>",
    "password": "<PASSWORD>"
}

In [4]:
WML_CREDENTIALS = {
                   "url": WOS_CREDENTIALS['url'],
                   "username": WOS_CREDENTIALS['username'],
                   "password": WOS_CREDENTIALS['password'],
                   "instance_id": "openshift",
                   "version" : "4.5" #If your env is CP4D 4.x.x then specify "4.x.x" instead of "4.5"  
                  }

In [5]:
#IBM DB2 database connection format example. This is required if you don't have any existing datamarts
DATABASE_CREDENTIALS = {
    "hostname":"***",
    "username":"***",
    "password":"***",
    "database":"***",
    "port":"***",
    "ssl":"***",
    "sslmode":"***",
    "certificate_base64":"***"}

### Action: put created schema name below.

In [6]:
#This is required if you don't have any existing datamarts
SCHEMA_NAME = '<SCHEMA_NAME>'

## Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Model building and deployment <a name="model"></a>

In this section you will learn how to train Spark MLLib model and next deploy it as web-service using Watson Machine Learning service.

## Load the training data from github

In [7]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv

--2024-09-06 09:33:16--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv


Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

german_credit_data_ 100%[===================>] 673.46K  --.-KB/s    in 0.02s   

2024-09-06 09:33:16 (36.1 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [8]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/06 09:33:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/09/06 09:33:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Row(CheckingStatus='0_to_200', LoanDuration=31, CreditHistory='credits_paid_to_date', LoanPurpose='other', LoanAmount=1889, ExistingSavings='100_to_500', EmploymentDuration='less_1', InstallmentPercent=3, Sex='female', OthersOnLoan='none', CurrentResidenceDuration=3, OwnsProperty='savings_insurance', Age=32, InstallmentPlans='none', Housing='own', ExistingCreditsCount=1, Job='skilled', Dependents=1, Telephone='none', ForeignWorker='yes', Risk='No Risk')

## Explore data

In [9]:
df_data.printSchema()

root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = true)



In [10]:
print("Number of records: " + str(df_data.count()))

Number of records: 5000


In [11]:
display(df_data)

DataFrame[CheckingStatus: string, LoanDuration: int, CreditHistory: string, LoanPurpose: string, LoanAmount: int, ExistingSavings: string, EmploymentDuration: string, InstallmentPercent: int, Sex: string, OthersOnLoan: string, CurrentResidenceDuration: int, OwnsProperty: string, Age: int, InstallmentPlans: string, Housing: string, ExistingCreditsCount: int, Job: string, Dependents: int, Telephone: string, ForeignWorker: string, Risk: string]

## Create a model

In [12]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Spark German Risk Model - AI Function"
DEPLOYMENT_NAME = "Spark German Risk Deployment - AI Function"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

Number of records for training: 4005


Number of records for evaluation: 995
root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = true)



The code below creates a Random Forest Classifier with Spark, setting up string indexers for the categorical features and the label column. Finally, this notebook creates a pipeline including the indexers and the model, and does an initial Area Under ROC evaluation of the model.

In [13]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml.feature import SQLTransformer

features = [x for x in spark_df.columns if x != 'Risk']
categorical_features = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']
categorical_num_features = [x + '_IX' for x in categorical_features]
si_list = [StringIndexer(inputCol=x, outputCol=y) for x, y in zip(categorical_features, categorical_num_features)]
va_features = VectorAssembler(inputCols=categorical_num_features + [x for x in features if x not in categorical_features], outputCol="features")

In [14]:
si_label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_label.labels)

In [15]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol="features")
pipeline = Pipeline(stages= si_list + [si_label, va_features, classifier, label_converter])

risk_model = pipeline.fit(train_data)

**Note**: If you want filter features from model output please replace `*` with feature names to be retained in `SQLTransformer` statement.

In [16]:
predictions = risk_model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderROC')
area_under_curve = evaluatorDT.evaluate(predictions)

evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderPR')
area_under_PR = evaluatorDT.evaluate(predictions)
#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve, "areaUnderPR = %g" % area_under_PR)

areaUnderROC = 0.724137 areaUnderPR = 0.680805


In [17]:
# extra code: evaluate more metrics by exporting them into pandas and numpy
from sklearn.metrics import classification_report
y_pred = predictions.toPandas()['prediction']
y_pred = ['Risk' if pred == 1.0 else 'No Risk' for pred in y_pred]
y_test = test_data.toPandas()['Risk']
print(classification_report(y_test, y_pred, target_names=['Risk', 'No Risk']))

24/09/06 09:33:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


              precision    recall  f1-score   support

        Risk       0.78      0.92      0.85       646
     No Risk       0.78      0.53      0.63       349

    accuracy                           0.78       995
   macro avg       0.78      0.72      0.74       995
weighted avg       0.78      0.78      0.77       995



## Save training data to Cloud Object Storage

## Cloud object storage details
In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit getting started with COS tutorial. You can find COS_API_KEY_ID and COS_RESOURCE_CRN variables in Service Credentials in menu of your COS instance. Used COS Service Credentials must be created with Role parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription.
COS_ENDPOINT variable can be found in Endpoint field of the menu.

In [18]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

In [19]:
COS_API_KEY_ID = "<COS_API_KEY>"
COS_RESOURCE_CRN = "<RESOURCE_INSTANCE_ID>"
COS_ENDPOINT = "<COS_ENDPOINT>" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [20]:
BUCKET_NAME = "<BUCKET_NAME>" #example: "credit-risk-training-data"

In [21]:
training_data_file_name="german_credit_data_biased_training.csv"

In [22]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [23]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

## Publish the model

In this section, the notebook uses Watson Machine Learning to save the model (including the pipeline) to the WML instance. Previous versions of the model are removed so that the notebook can be run again, resetting all data for another demo.

In [24]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.360'

In [25]:
space_name = "<SPACE_NAME>"
# create the space and set it as default
space_meta_data = {
        wml_client.spaces.ConfigurationMetaNames.NAME : space_name,
        wml_client.spaces.ConfigurationMetaNames.DESCRIPTION : 'tutorial_space'
}
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["id"]
if space_id is None:
    space_id = wml_client.spaces.store(meta_props=space_meta_data)["metadata"]["id"]

print(space_id)
wml_client.set.default_space(space_id)

02f94eae-b785-4f2b-a8c8-4d28a3dd0273


'SUCCESS'

### Remove existing model and deployment

In [26]:
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

Deleting deployment id a73a0a99-df71-48df-baf6-868dc331f5cb
Deleting model id ede5f782-b3c1-44d6-a469-3d2611e775fa


--  ----  -------  ----  ----------  ----------------
ID  NAME  CREATED  TYPE  SPEC_STATE  SPEC_REPLACEMENT
--  ----  -------  ----  ----------  ----------------


,ID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT


#### Add training data reference either from DB2 on CP4D or Cloud Object Storage

In [27]:
# COS training data reference example format
datasource_type = wml_client.connections.get_datasource_type_uid_by_name('bluemixcloudobjectstorage')
conn_meta_props= {
    wml_client.connections.ConfigurationMetaNames.NAME: "Connection My COS ",
    wml_client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: datasource_type,
    wml_client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to my COS",
    wml_client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': BUCKET_NAME,
        'api_key': COS_API_KEY_ID,
        'resource_instance_id': COS_RESOURCE_CRN,
        'iam_url': "https://iam.ng.bluemix.net/oidc/token",
        'url': COS_ENDPOINT
    }
}

conn_details = wml_client.connections.create(meta_props=conn_meta_props)
connection_id = wml_client.connections.get_uid(conn_details)

training_data_references = [
    {
        "id": "German Credit Risk", 
        "type": "connection_asset",
        "connection": {
            "id": connection_id,
            "href": "/v2/connections/" + connection_id + "?space_id=" + space_id

        },
        "location": {
            "bucket": BUCKET_NAME,
            "file_name": training_data_file_name
        }
    }    
]


Creating connections...


SUCCESS


In [28]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_3.4")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
        wml_client._models.ConfigurationMetaNames.TYPE: "mllib_3.4",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "Risk",
    }

Software Specification ID: dabb5e1b-e1c7-5ef5-95ea-34a8223e436c


In [29]:
print("Storing model ...")
published_model_details = wml_client.repository.store_model(
    model=risk_model, 
    meta_props=model_props, 
    training_data=train_data, 
    pipeline=pipeline)

model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...


Done
Model ID: 4a60bd7f-9640-411f-8791-7cb4c56ff918


In [30]:
wml_client.repository.list_models()

------------------------------------  -------------------------------------  ------------------------  ---------  ----------  ----------------
ID                                    NAME                                   CREATED                   TYPE       SPEC_STATE  SPEC_REPLACEMENT
4a60bd7f-9640-411f-8791-7cb4c56ff918  Spark German Risk Model - AI Function  2024-09-06T09:34:17.002Z  mllib_3.4  supported
------------------------------------  -------------------------------------  ------------------------  ---------  ----------  ----------------


,ID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,4a60bd7f-9640-411f-8791-7cb4c56ff918,Spark German Risk Model - AI Function,2024-09-06T09:34:17.002Z,mllib_3.4,supported,


## Deploy the model

The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [31]:
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(DEPLOYMENT_NAME),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid=wml_client.deployments.get_id(deployment_details)

print("Scoring URL:" + scoring_url)
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))



#######################################################################################

Synchronous deployment creation for uid: '4a60bd7f-9640-411f-8791-7cb4c56ff918' started

#######################################################################################


initializing


Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='7c9082df-99ef-4165-90c5-92773bc740a7'
------------------------------------------------------------------------------------------------


Scoring URL:https://cpd-cpd-instance.apps.wxg415nfs2860.cp.fyre.ibm.com/ml/v4/deployments/7c9082df-99ef-4165-90c5-92773bc740a7/predictions
Model id: 4a60bd7f-9640-411f-8791-7cb4c56ff918
Deployment id: 7c9082df-99ef-4165-90c5-92773bc740a7


## Define AI Function

In [32]:
ai_params = {"wml_credentials": WML_CREDENTIALS, 
             "deployment_uid": deployment_uid,
             "space_id": space_id
            }

In [33]:
#AI function definition
def score_generator(params=ai_params):
    
    import json
    from ibm_watson_machine_learning import APIClient
   
    wml_credentials = params["wml_credentials"]
    deployment_uid = params["deployment_uid"]
    space_id = params["space_id"]

    client = APIClient(wml_credentials)
    client.set.default_space(space_id)

    def score(payload):
        scores_area = client.deployments.score(deployment_uid, payload)
        return scores_area
    return score

In [34]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"]
]
sample_payload = { "input_data": [ {"fields": fields,"values": values}]}

score = score_generator()
scores_ai = score(sample_payload)
wml_client.set.default_space(space_id)
print(scores_ai)





{'predictions': [{'fields': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'LoanPurpose_IX', 'ExistingSavings_IX', 'EmploymentDuration_IX', 'Sex_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'InstallmentPlans_IX', 'Housing_IX', 'Job_IX', 'Telephone_IX', 'ForeignWorker_IX', 'label', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, [20, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19], [1.0, 1.0, 1.0, 13.

In [35]:
#Store the function
func_name = 'Credit Risk python Fn Model'
meta_data = {
    wml_client.repository.FunctionMetaNames.NAME: func_name,
    #Note if there is specification related exception then use "default_py3.7_opence" instead of default_py3.8
    wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: wml_client.software_specifications.get_id_by_name("runtime-23.1-py3.10")
}

function_details = wml_client.repository.store_function(meta_props=meta_data, function=score_generator)

In [36]:
function_details
ai_function_uid = function_details['metadata']['id']

In [37]:
#Generate the deployment
function_deployment_details = wml_client.deployments.create(artifact_uid=ai_function_uid, meta_props={wml_client.deployments.ConfigurationMetaNames.NAME: 'dep_' + func_name,wml_client.deployments.ConfigurationMetaNames.ONLINE: {}})
ai_func_deployment_uid = wml_client.deployments.get_uid(function_deployment_details)
print("AI Function Deployment UID:" + ai_func_deployment_uid)
scoring_url = function_deployment_details["entity"]["status"]["online_url"]["url"]
print(scoring_url)



#######################################################################################

Synchronous deployment creation for uid: '59f45b49-5faa-4546-8e8c-8fad77fbb351' started

#######################################################################################


initializing


Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.

.

.

.

.

.

.


ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b51ed35d-58a5-4def-8b18-ca3d83185a38'
------------------------------------------------------------------------------------------------


AI Function Deployment UID:b51ed35d-58a5-4def-8b18-ca3d83185a38
https://cpd-cpd-instance.apps.wxg415nfs2860.cp.fyre.ibm.com/ml/v4/deployments/b51ed35d-58a5-4def-8b18-ca3d83185a38/predictions


# Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [38]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

In [39]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )
instance_id='<SERVICE_INSTANCE_ID' #Datamart id
wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator,service_instance_id = instance_id)
wos_client.version

'3.0.40'

## Create datamart

### Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were supplied, the datamart will be created there unless there is an existing datamart and the KEEP_MY_INTERNAL_POSTGRES variable is set to True. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the German Credit model will be removed from OpenScale monitoring.

In [40]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-09-04 19:53:05.982000+00:00,00000000-0000-0000-0000-000000000000


In [41]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.DB2,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DATABASE_CREDENTIALS['hostname'],
                        username=DATABASE_CREDENTIALS['username'],
                        password=DATABASE_CREDENTIALS['password'],
                        db=DATABASE_CREDENTIALS['database'],
                        port=DATABASE_CREDENTIALS['port'],
                        ssl=DATABASE_CREDENTIALS['ssl'],
                        sslmode=DATABASE_CREDENTIALS['sslmode'],
                        certificate_base64=DATABASE_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 00000000-0000-0000-0000-000000000000


## Remove existing service provider connected with used WML instance.
Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one.

In [42]:
SERVICE_PROVIDER_NAME = "WML AI function -  WOS notebook"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WML AI function -  WOS notebook."

In [43]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

Deleted existing service_provider for WML instance: 1ad410ae-cffa-4f7e-9a96-e8c57a603411


## Add service provider
Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

**Note:** You can bind more than one engine instance if needed by calling `wos_client.service_providers.add` method. Next, you can refer to particular service provider using `service_provider_id`.

In [44]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = space_id,
        operational_space_id = "production",
        credentials=WMLCredentialsCP4D(),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider 58d44bfd-b74b-478d-a4ab-5f59cce561f4 






active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [45]:
wos_client.service_providers.show()

99999999-9999-9999-9999-999999999999,active,WML AI function - WOS notebook,watson_machine_learning,2024-09-06 09:35:38.656000+00:00,58d44bfd-b74b-478d-a4ab-5f59cce561f4
99999999-9999-9999-9999-999999999999,active,WOS ExpressPath WML pre_production binding,watson_machine_learning,2024-09-04 19:53:24.059000+00:00,8a8b7e3c-c1b8-427a-8b07-3b62208bd1e7
99999999-9999-9999-9999-999999999999,active,WOS ExpressPath WML production binding,watson_machine_learning,2024-09-04 19:53:17.989000+00:00,3a0b387c-2b4d-42ea-936a-dd875069b717


In [46]:
print(deployment_uid)
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id,deployment_id=ai_func_deployment_uid, deployment_space_id = space_id).result['resources'][0]
asset_deployment_details

7c9082df-99ef-4165-90c5-92773bc740a7


{'metadata': {'guid': 'b51ed35d-58a5-4def-8b18-ca3d83185a38',
  'created_at': '2024-09-06T09:34:36.066Z',
  'modified_at': '2024-09-06T09:34:36.066Z'},
 'entity': {'name': 'dep_Credit Risk python Fn Model',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/b51ed35d-58a5-4def-8b18-ca3d83185a38/predictions'},
  'asset': {},
  'asset_properties': {}}}

In [47]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=ai_func_deployment_uid,deployment_space_id=space_id)
model_asset_details_from_deployment

{'metadata': {'guid': 'b51ed35d-58a5-4def-8b18-ca3d83185a38',
  'created_at': '2024-09-06T09:34:36.066Z',
  'modified_at': '2024-09-06T09:34:36.066Z'},
 'entity': {'name': 'dep_Credit Risk python Fn Model',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/b51ed35d-58a5-4def-8b18-ca3d83185a38/predictions'},
  'asset': {'asset_id': '59f45b49-5faa-4546-8e8c-8fad77fbb351',
   'url': 'https://internal-nginx-svc:12443/ml/v4/functions/59f45b49-5faa-4546-8e8c-8fad77fbb351?space_id=02f94eae-b785-4f2b-a8c8-4d28a3dd0273&version=2020-06-12',
   'name': 'Credit Risk python Fn Model',
   'asset_type': 'function',
   'created_at': '2024-09-06T09:34:35.543Z',
   'modified_at': '2024-09-06T09:34:35.851Z'},
  'asset_properties': {'model_type': 'python',
   'runtime_environment': 'python-3.10'}}}

## Subscriptions

### Remove existing credit risk subscriptions

This code removes previous subscriptions to the German Credit model to refresh the monitors with the new model and new data.

In [48]:
wos_client.subscriptions.show()

3e870a0e-118e-47a0-b290-8f60ac61c09d,model,GermanCreditRiskModelICP,00000000-0000-0000-0000-000000000000,e207a166-3ca2-4886-9f1f-8428ba70d2b6,GermanCreditRiskModelICP,3a0b387c-2b4d-42ea-936a-dd875069b717,active,2024-09-04 20:09:30.886000+00:00,54761e81-a87e-4ff6-b3f0-7b368ef6c392
39ae600e-5a0c-426c-91e2-17dcf8be8bec,model,GermanCreditRiskModelPreProdICP,00000000-0000-0000-0000-000000000000,32b80eb6-c461-4b3a-9345-51d3fea01e64,GermanCreditRiskModelPreProdICP,8a8b7e3c-c1b8-427a-8b07-3b62208bd1e7,active,2024-09-04 20:07:27.645000+00:00,dd7346f2-7346-4303-9295-fd822173ded6
d98ac736-c7f2-41ee-87db-a13438634bec,model,GermanCreditRiskModelChallengerICP,00000000-0000-0000-0000-000000000000,214a512f-1204-40cd-a1c1-e098b673fdd0,GermanCreditRiskModelChallengerICP,8a8b7e3c-c1b8-427a-8b07-3b62208bd1e7,active,2024-09-04 20:03:55.397000+00:00,5d0a8def-7048-4a11-87b4-b9a33fd34bf9


In [49]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == ai_function_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [50]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='Risk',
            probability_fields=['probability'],
            prediction_field='predictedLabel',
            feature_fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
            categorical_fields = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
            training_data_reference=TrainingDataReference(type='cos',
                                                          location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                                    file_name = training_data_file_name),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": COS_RESOURCE_CRN,
                                                                        "url": COS_ENDPOINT,
                                                                        "api_key": COS_API_KEY_ID,
                                                                        "iam_url": IAM_URL})),
            training_data_schema=None
        ),
        background_mode=False
    ).result
subscription_id = subscription_details.metadata.id
subscription_id




 Waiting for end of adding subscription 41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac 






active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------




'41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac'

In [51]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  d413e6bc-eb6c-4e37-8124-efe5d9de3927


In [52]:
wos_client.data_sets.show()

00000000-0000-0000-0000-000000000000,active,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac,subscription,payload_logging_error,2024-09-06 09:35:46.372000+00:00,2c2d16d4-0da9-4632-9823-326aa0eb4596
00000000-0000-0000-0000-000000000000,active,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac,subscription,model_health,2024-09-06 09:35:46.095000+00:00,4cb39a46-5406-4848-9387-9d81bbcb05a8
00000000-0000-0000-0000-000000000000,active,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac,subscription,manual_labeling,2024-09-06 09:35:46.332000+00:00,d24e8ba6-59e2-4070-9b69-e355f122115b
00000000-0000-0000-0000-000000000000,active,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac,subscription,payload_logging,2024-09-06 09:35:46.281000+00:00,d413e6bc-eb6c-4e37-8124-efe5d9de3927
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,model_health,2024-09-04 20:09:32.543000+00:00,1f539a49-c11f-4431-a74f-544d95f289b5
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,manual_labeling,2024-09-04 20:09:33.664000+00:00,92e843a0-7b93-4faa-9b7a-a7982644e2df
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,payload_logging,2024-09-04 20:09:33.439000+00:00,f10f4d04-2231-41e1-ba5e-b128e11d4962
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,feedback,2024-09-04 20:10:43.375000+00:00,48c2cbc1-bd5a-48d7-9a59-5f99a5ac05e5
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,drift_intervals,2024-09-04 20:10:02.344000+00:00,4878bc77-beeb-41d9-a21e-e8ce90fcd3a1
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,drift_insights,2024-09-04 20:10:02.348000+00:00,519c28fe-a19d-4312-8906-b82bd4bf39ec


Note: First 10 records were displayed.


### Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model.

In [53]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = { "input_data": [ {"fields": fields,"values": values}]}
scoring_response = wml_client.deployments.score(ai_func_deployment_uid, payload_scoring)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

Single record scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'LoanPurpose_IX', 'ExistingSavings_IX', 'EmploymentDuration_IX', 'Sex_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'InstallmentPlans_IX', 'Housing_IX', 'Job_IX', 'Telephone_IX', 'ForeignWorker_IX', 'label', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, [20, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19], [1.0, 1.

## Check if WML payload logging worked else manually store payload records

In [54]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": scoring_response['predictions'][0]['fields'], "values":scoring_response['predictions'][0]['values']},
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

Number of records in the payload logging table: 8



# Quality monitoring and feedback logging <a name="quality"></a>

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [55]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation 03e0da1a-eac1-430f-ae4a-a3ac140baa7c 






preparing


active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [56]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'03e0da1a-eac1-430f-ae4a-a3ac140baa7c'

## Feedback logging

The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.

In [57]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/additional_feedback_data_v2.json

--2024-09-06 09:36:22--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/additional_feedback_data_v2.json


Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 50890 (50K) [text/plain]
Saving to: ‘additional_feedback_data_v2.json’

additional_feedback 100%[===================>]  49.70K  --.-KB/s    in 0.004s  

2024-09-06 09:36:22 (12.5 MB/s) - ‘additional_feedback_data_v2.json’ saved [50890/50890]



## Get feedback logging dataset ID

In [58]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

{
  "data_sets": [
    {
      "metadata": {
        "id": "b0cb45b5-cd64-4056-ab75-7e113daaa524",
        "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:data_set:b0cb45b5-cd64-4056-ab75-7e113daaa524",
        "url": "/v2/data_sets/b0cb45b5-cd64-4056-ab75-7e113daaa524",
        "created_at": "2024-09-06T09:36:12.342000Z",
        "created_by": "internal-service",
        "modified_at": "2024-09-06T09:36:12.655000Z",
        "modified_by": "internal-service"
      },
      "entity": {
        "data_mart_id": "00000000-0000-0000-0000-000000000000",
        "name": "41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac_feedback",
        "description": "41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac_feedback",
        "type": "feedback",
        "target": {
          "target_type": "subscription",
          "target_id": "41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac"
        },
        "schema_update_mode": "auto",
        "data_schema": {
          "type": "struct",
          "

In [59]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

In [60]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)




 Waiting for end of storing records with request id: c7b8691f-2f69-4a45-8d76-a85a745ee63b 






active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [61]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

98

In [62]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 1f684381-3ec4-4138-9e8d-ff31eaadbe14 






running

.

.

.

...

.

.

.

.


finished

---------------------------
 Successfully finished run 
---------------------------




In [63]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-09-06 09:36:28.860000+00:00,true_positive_rate,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.36363636363636365,None,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:36:28.860000+00:00,area_under_roc,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.6510489510489511,0.8,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:36:28.860000+00:00,precision,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.75,None,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:36:28.860000+00:00,matthews_correlation_coefficient,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.38627307784817894,None,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:36:28.860000+00:00,f1_measure,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.48979591836734687,None,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:36:28.860000+00:00,accuracy,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.7448979591836735,None,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:36:28.860000+00:00,label_skew,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.6909336273400492,None,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:36:28.860000+00:00,gini_coefficient,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.3020979020979022,None,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:36:28.860000+00:00,log_loss,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.4657072128041715,None,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:36:28.860000+00:00,false_positive_rate,cb3cd6de-2fc7-45c8-878f-01cc6c5b9836,0.06153846153846154,None,None,['model_type:original'],quality,03e0da1a-eac1-430f-ae4a-a3ac140baa7c,1f684381-3ec4-4138-9e8d-ff31eaadbe14,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac


Note: First 10 records were displayed.


# Fairness, drift monitoring and explanations 
 <a name="fairness"></a>

The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 200 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [64]:
wos_client.monitor_instances.show()

00000000-0000-0000-0000-000000000000,active,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac,subscription,quality,2024-09-06 09:36:12.075000+00:00,03e0da1a-eac1-430f-ae4a-a3ac140baa7c
00000000-0000-0000-0000-000000000000,active,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac,subscription,model_health,2024-09-06 09:35:45.740000+00:00,257abea9-751c-45d0-bfac-5b36401bb49f
00000000-0000-0000-0000-000000000000,active,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac,subscription,performance,2024-09-06 09:35:46.023000+00:00,d9cfd00b-ff85-42e0-b734-ddd494749591
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,model_health,2024-09-04 20:09:32.132000+00:00,a66570b6-af7e-42be-97fe-61d71e2844e5
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,mrm,2024-09-04 20:10:52.899000+00:00,372f4893-e757-4019-8cd8-f8bb893c1e64
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,fairness,2024-09-04 20:10:31.948000+00:00,9d5bf027-400e-4069-8382-7575091ed40d
00000000-0000-0000-0000-000000000000,active,b1bc49e2-49d4-4e60-ab73-104f9c07bf9f,instance,performance,2024-09-06 08:34:30.022000+00:00,9b90ca6c-894d-4e7d-82a5-c97fab71aa4c
00000000-0000-0000-0000-000000000000,active,e8095b6b-6d85-46d6-bce8-1f6406be4eea,instance,performance,2024-09-06 07:07:35.281000+00:00,11027f78-d778-49fa-8f5f-24f02ca459c5
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,drift_v2,2024-09-04 20:10:01.326000+00:00,f9152979-973c-49b5-8177-6b89babe1554
00000000-0000-0000-0000-000000000000,active,54761e81-a87e-4ff6-b3f0-7b368ef6c392,subscription,quality,2024-09-04 20:10:42.470000+00:00,0aefb57e-029a-43c3-82ea-54049ee1702a


Note: First 10 records were displayed.


In [65]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id




 Waiting for end of monitor instance creation 94691fda-994c-478e-b3fb-7a03f1de4d0c 






active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'94691fda-994c-478e-b3fb-7a03f1de4d0c'

## Drift configuration

In [66]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [67]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 10,
    "drift_threshold": 0.1,
    "train_drift_model": True,
    "enable_model_drift": False,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id




 Waiting for end of monitor instance creation 5c0fb7a3-4128-4d42-bf76-ac2579b8efd7 






active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'5c0fb7a3-4128-4d42-bf76-ac2579b8efd7'

#### Finding feature importances

This is done because we will deduce the important and most important features to the model. This will help in narrowing down the analysis of Drift evaluation in the UI

In [68]:
feature_importance = dict(zip(features, risk_model.stages[-2].featureImportances))

## Drift V2 Configuration

In [69]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift_v2" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift v2 monitor instance with id: ', monitor_instance.metadata.id)

In [70]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
        "min_samples": 100,
        "max_samples": 1000,
        "train_archive": True,
        "features": {
            "fields": fields,
            "importances": feature_importance
        }
    }
drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

drift_v2_monitor_instance_id = drift_v2_monitor_details.metadata.id
drift_v2_monitor_instance_id

'35c5398c-3f0e-45cc-beba-63f3fe291b34'

## Score the model again now that monitoring is configured

This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness.

In [71]:
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_feed.json
!ls -lh german_credit_feed.json

--2024-09-06 09:37:31--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_feed.json


Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 3076279 (2.9M) [text/plain]
Saving to: ‘german_credit_feed.json.6’

          german_cr   0%[                    ]       0  --.-KB/s               

german_credit_feed. 100%[===================>]   2.93M  --.-KB/s    in 0.03s   

2024-09-06 09:37:31 (102 MB/s) - ‘german_credit_feed.json.6’ saved [3076279/3076279]



-rw-rw----. 1 1000750000 wscommon 3.0M Sep  6 07:00 german_credit_feed.json


Score 200 randomly chosen records

In [72]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

scoring_response = wml_client.deployments.score(ai_func_deployment_uid, payload_scoring)
time.sleep(5)

if pl_records_count == 8:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response=scoring_response,
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

Payload logging did not happen, performing explicit payload logging.


Number of records in the payload logging table: 408


In [73]:
print('Number of records in payload table: ', wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id))

Number of records in payload table:  408


## Run fairness monitor

Kick off a fairness monitor run on current data. The monitor runs hourly, but can be manually initiated using the Python client, the REST API, or the graphical user interface.

In [74]:
time.sleep(5)
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 5d4a81f0-b3cb-4383-b3b8-992183b0e48c 






running


finished

---------------------------
 Successfully finished run 
---------------------------




In [75]:
time.sleep(10)
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-09-06 09:37:57.336252+00:00,fairness_value,7fa2419c-f992-4603-b026-98c9282e2d0c,98.75,80.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,94691fda-994c-478e-b3fb-7a03f1de4d0c,5d4a81f0-b3cb-4383-b3b8-992183b0e48c,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:37:57.336252+00:00,fairness_value,7fa2419c-f992-4603-b026-98c9282e2d0c,105.0,80.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,94691fda-994c-478e-b3fb-7a03f1de4d0c,5d4a81f0-b3cb-4383-b3b8-992183b0e48c,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac


## Run drift monitor

Kick off a drift monitor run on current data. The monitor runs every hour, but can be manually initiated using the Python client, the REST API.

In [76]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run b6e502c9-dddd-4eaf-b084-1ae4a286f127 






finished

---------------------------
 Successfully finished run 
---------------------------




In [77]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-09-06 09:38:09.670474+00:00,data_drift_magnitude,e7ee4169-8992-4dc6-b802-0427250a0948,0.08333333333333333,None,0.1,[],drift,5c0fb7a3-4128-4d42-bf76-ac2579b8efd7,b6e502c9-dddd-4eaf-b084-1ae4a286f127,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac


## Run drift v2 monitor


Kick off a drift v2 monitor run on current data. The monitor runs every day, but can be manually initiated using the Python client, the REST API.

In [78]:
drift_v2_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_v2_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 8ab24475-05a5-403e-a79f-584ba1ad66b8 





.

.

.

.

.

.

.


finished

---------------------------
 Successfully finished run 
---------------------------




In [79]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_v2_monitor_instance_id)

2024-09-06 09:40:13.217206+00:00,records_processed,67e9bb17-87f8-4bad-a968-92a9ecbf6984,20.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:40:13.217206+00:00,confidence_drift_score,67e9bb17-87f8-4bad-a968-92a9ecbf6984,0.2921,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:40:13.217206+00:00,records_processed,67e9bb17-87f8-4bad-a968-92a9ecbf6984,20.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:40:13.217206+00:00,confidence_drift_score,67e9bb17-87f8-4bad-a968-92a9ecbf6984,0.2923,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:40:17.910469+00:00,records_processed,ffb6dbcc-c960-42e7-b9a3-9a691c4a22dc,20.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:40:17.910469+00:00,confidence_drift_score,ffb6dbcc-c960-42e7-b9a3-9a691c4a22dc,0.3252,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:40:17.910469+00:00,records_processed,ffb6dbcc-c960-42e7-b9a3-9a691c4a22dc,20.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:40:17.910469+00:00,confidence_drift_score,ffb6dbcc-c960-42e7-b9a3-9a691c4a22dc,0.3257,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:40:21.173355+00:00,records_processed,76a3a302-9ac3-4a00-9273-672cd44f3294,20.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:predictedLabel']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac
2024-09-06 09:40:21.173355+00:00,prediction_drift_score,76a3a302-9ac3-4a00-9273-672cd44f3294,0.0,None,0.05,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:predictedLabel']",drift_v2,35c5398c-3f0e-45cc-beba-63f3fe291b34,8ab24475-05a5-403e-a79f-584ba1ad66b8,subscription,41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac


Note: First 10 records were displayed.


## Configure Explainability

Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [80]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation 5ecf3208-a19b-4106-9546-7d1e747137fd 






preparing

.


active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Run explanation for sample record

In [81]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result
print(result)

Running explanations on scoring IDs: ['c9876b10-f190-475e-b570-829e7266231d-1']


{
  "metadata": {
    "explanation_task_ids": [
      "00848c40-dd95-4b9f-ac5c-5441d7df5c6f"
    ],
    "created_by": "1000331001",
    "created_at": "2024-09-06T09:46:08.976603Z"
  }
}


# Custom monitors and metrics <a name="custom"></a>

## Register custom monitor

In [82]:
def get_definition(monitor_name):
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    
    for definition in monitor_definitions:
        if monitor_name == definition.entity.name:
            return definition
    
    return None

In [83]:
monitor_name = 'my model performance'
metrics = [MonitorMetricRequest(name='sensitivity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)]),
          MonitorMetricRequest(name='specificity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.75)])]
tags = [MonitorTagRequest(name='region', description='customer geographical region')]

existing_definition = get_definition(monitor_name)

if existing_definition is None:
    custom_monitor_details = wos_client.monitor_definitions.add(name=monitor_name, metrics=metrics, tags=tags, background_mode=False).result
else:
    custom_monitor_details = existing_definition

## Show available monitors types

In [84]:
wos_client.monitor_definitions.show()

my_model_performance,my model performance,"['sensitivity', 'specificity']"
fairness,Fairness,"['Disparate impact', 'Average odds difference', 'False discovery rate difference', 'Error rate difference', 'False negative rate difference', 'False omission rate difference', 'False positive rate difference', 'True positive rate difference', 'Average absolute odds difference', 'Statistical parity difference', 'Impact score']"
model_health,Model health,"['Total scoring requests', 'Total records', 'Average records', 'Median records', 'Maximum records', 'Minimum records', 'Total payload size', 'Average payload size', 'Median payload size', 'Minimum payload size', 'Maximum payload size', 'Average API throughput', 'Minimum API throughput', 'Maximum API throughput', 'Median API throughput', 'Average API latency', 'Minimum API latency', 'Maximum API latency', 'Median API latency', 'Average record throughput', 'Minimum record throughput', 'Maximum record throughput', 'Median record throughput', 'Average record latency', 'Minimum record latency', 'Maximum record latency', 'Median record latency', 'Users', 'Errors', 'Data errors', 'System errors', 'Total input token count', 'Average input token count', 'Median input token count', 'Maximum input token count', 'Minimum input token count', 'Total output token count', 'Average output token count', 'Median output token count', 'Maximum output token count', 'Minimum output token count']"
performance,Performance,['Number of records']
data_health,Data Health,"['Absence Count', 'Empty Strings', 'Data Type Mismatch', 'Class Confusion', 'Duplicate Rows', 'Unique Columns']"
explainability,Explainability,['Global explanation stability']
mrm,Model risk management,"['Tests run', 'Tests passed', 'Tests failed', 'Tests skipped', 'Fairness score', 'Quality score', 'Drift score']"
generative_ai_quality,Generative AI Quality,"['Records processed', 'Faithfulness', 'Answer relevance', 'Unsuccessful requests', 'Answer similarity', 'Context relevance', 'Retrieval Precision', 'Average Precision', 'Reciprocal Rank', 'Hit rate', 'Normalized Discounted Cumulative Gain', 'Coverage', 'Density', 'Compression', 'Repetitiveness', 'Abstractness', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-Lsum', 'SARI', 'METEOR', 'F1 Score', 'Precision', 'Recall', 'BLEU', 'Jaccard similarity', 'Cosine similarity', 'Output data PII', 'Input data PII', 'Output data HAP', 'Input data HAP', 'Readability', 'Micro F1 Score', 'Macro F1 Score', 'Micro precision', 'Macro precision', 'Micro recall', 'Macro recall', 'Exact match']"
drift_v2,Drift v2,"['Embedding drift', 'Output drift', 'Prediction drift', 'Model quality drift', 'Feature drift', 'Input metadata drift', 'Output metadata drift', 'Records processed', 'New records processed']"
quality,Quality,"['Area under ROC', 'Area under PR', 'Proportion explained variance', 'Mean absolute error', 'Mean squared error', 'R squared', 'Root of mean squared error', 'Accuracy', 'Weighted True Positive Rate (wTPR)', 'True positive rate (TPR)', 'Weighted False Positive Rate (wFPR)', 'False positive rate (FPR)', 'Weighted recall', 'Recall', 'Weighted precision', 'Precision', 'Weighted F1-Measure', 'F1-Measure', 'Logarithmic loss', 'Brier score', 'Matthews correlation coefficient', 'Label skew', 'Gini coefficient', 'Pearson', 'Spearman', 'Mean absolute percentage error', 'Symmetric mean absolute percentage error']"


Note: First 10 records were displayed.


### Get monitors uids and details

In [85]:
custom_monitor_id = custom_monitor_details.metadata.id

print(custom_monitor_id)

my_model_performance


In [86]:
custom_monitor_details = wos_client.monitor_definitions.get(monitor_definition_id=custom_monitor_id).result
print('Monitor definition details:', custom_monitor_details)

Monitor definition details: {
  "metadata": {
    "id": "my_model_performance",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_definition:my_model_performance",
    "url": "/v2/monitor_definitions/my_model_performance",
    "created_at": "2024-09-06T07:07:05.355000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "my model performance",
    "metrics": [
      {
        "name": "sensitivity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.8
          }
        ],
        "expected_direction": "increasing",
        "id": "sensitivity"
      },
      {
        "name": "specificity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.75
          }
        ],
        "expected_direction": "increasing",
        "id": "specificity"
      }
    ],
    "tags": [
      {
        "name": "region",
        "description": "customer geo

## Enable custom monitor for subscription

In [87]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
    )

thresholds = [MetricThresholdOverride(metric_id='sensitivity', type = MetricThresholdTypes.LOWER_LIMIT, value=0.9)]

parameters = dict()

custom_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=custom_monitor_id,
            target=target,
            thresholds=thresholds,
            parameters = parameters
).result




 Waiting for end of monitor instance creation e243a327-a471-474a-b38e-3ca9c702c6b3 






active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Get monitor instance id and configuration details

In [88]:
custom_monitor_instance_id = custom_monitor_instance_details.metadata.id

In [89]:
custom_monitor_instance_details = wos_client.monitor_instances.get(custom_monitor_instance_id).result
print(custom_monitor_instance_details)

{
  "metadata": {
    "id": "e243a327-a471-474a-b38e-3ca9c702c6b3",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:e243a327-a471-474a-b38e-3ca9c702c6b3",
    "url": "/v2/monitor_instances/e243a327-a471-474a-b38e-3ca9c702c6b3",
    "created_at": "2024-09-06T09:46:10.235000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-09-06T09:46:10.283000Z",
    "modified_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "my_model_performance",
    "target": {
      "target_type": "subscription",
      "target_id": "41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac"
    },
    "parameters": {},
    "thresholds": [
      {
        "metric_id": "sensitivity",
        "type": "lower_limit",
        "value": 0.9
      }
    ],
    "status": {
      "state": "active"
    }
  }
}


## Storing custom metrics

In [90]:
from datetime import datetime, timezone, timedelta
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
measurement_request = [MonitorMeasurementRequest(timestamp=datetime.now(timezone.utc), 
                                                 metrics=[{"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}])]
print(measurement_request[0])

{
  "timestamp": "2024-09-06T09:46:15.660958Z",
  "metrics": [
    {
      "specificity": 0.78,
      "sensitivity": 0.67,
      "region": "us-south"
    }
  ]
}


In [91]:
published_measurement_response = wos_client.monitor_instances.measurements.add(
    monitor_instance_id=custom_monitor_instance_id,
    monitor_measurement_request=measurement_request).result
published_measurement_id = published_measurement_response[0]["measurement_id"]
print(published_measurement_response)

[{'measurement_id': 'f863b63b-5fb6-497f-a689-6903ffd15c73', 'metrics': [{'region': 'us-south', 'sensitivity': 0.67, 'specificity': 0.78}], 'timestamp': '2024-09-06T09:46:15.660958Z'}]


### List and get custom metrics

In [92]:
time.sleep(5)
published_measurement = wos_client.monitor_instances.measurements.get(monitor_instance_id=custom_monitor_instance_id, measurement_id=published_measurement_id).result
print(published_measurement)

{
  "metadata": {
    "id": "f863b63b-5fb6-497f-a689-6903ffd15c73",
    "crn": "",
    "url": "/v2/monitor_instances/e243a327-a471-474a-b38e-3ca9c702c6b3/measurements/f863b63b-5fb6-497f-a689-6903ffd15c73",
    "created_at": "2024-09-06T09:46:15.857681Z",
    "created_by": "N/A"
  },
  "entity": {
    "timestamp": "2024-09-06T09:46:15.660958Z",
    "values": [
      {
        "metrics": [
          {
            "id": "sensitivity",
            "value": 0.67,
            "lower_limit": 0.9
          },
          {
            "id": "specificity",
            "value": 0.78
          }
        ],
        "tags": [
          {
            "id": "region",
            "value": "us-south"
          }
        ]
      }
    ],
    "issue_count": 1,
    "target": {
      "target_type": "subscription",
      "target_id": "41b7c70a-5105-4b0e-9aa5-9a7b43fa94ac"
    },
    "monitor_instance_id": "e243a327-a471-474a-b38e-3ca9c702c6b3",
    "monitor_definition_id": "my_model_performance"
  }
}


# Historical data <a name="historical"></a>

In [93]:
historyDays = 7

 ## Insert historical payloads

The next section of the notebook downloads and writes historical data to the payload and measurement tables to simulate a production model that has been monitored and receiving regular traffic for the last seven days. This historical data can be viewed in the Watson OpenScale user interface. The code uses the Python and REST APIs to write this data.

In [94]:
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_fairness_v2.json
!ls -lh history_fairness_v2.json

--2024-09-06 09:46:21--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_fairness_v2.json


Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 1186317 (1.1M) [text/plain]
Saving to: ‘history_fairness_v2.json.2’

          history_f   0%[                    ]       0  --.-KB/s               

         history_fa  29%[====>               ] 344.00K  1.67MB/s               

        history_fai  54%[=========>          ] 625.81K  1.47MB/s               

history_fairness_v2 100%[===================>]   1.13M  2.10MB/s    in 0.5s    

2024-09-06 09:46:22 (2.10 MB/s) - ‘history_fairness_v2.json.2’ saved [1186317/1186317]



-rw-rw----. 1 1000750000 wscommon 1.2M Sep  6 07:07 history_fairness_v2.json


In [95]:
from datetime import datetime, timedelta, timezone
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Source
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Measurements
with open("history_fairness_v2.json") as f:
    fairness_values = json.load(f)
    for day in range(historyDays):
        print('Loading day', day + 1)
        daily_measurement_requests = []
        sources_list = []
        for hour in range(24):
            score_time = (datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
            index = (day * 24 + hour) % len(fairness_values) # wrap around and reuse values if needed
            fairness_values[index]["timestamp"] = score_time
            #print(score_time) 
            fairness_value = fairness_values[index]
            metrics_list = fairness_value["metrics"]
            sources = fairness_value["sources"]
            sources_list = []
            for source in sources:
                source_id = source["id"]
                source_type = source["type"]
                source_data = source["data"]
                if source_id == "bias_detection_summary":
                    source_data["evaluated_at"] = score_time
                    source_data["favourable_class"] = ["No Risk"]
                    source_data["unfavourable_class"] = ["Risk"]
                    source_data["score_type"] = "disparate impact"
                sources_list.append(
                    Source(
                        id=source_id,
                        type=source_type,
                        data=source_data
                    )
                )  
            measurement_request = MonitorMeasurementRequest(metrics=metrics_list, sources=sources_list, timestamp=score_time)
            daily_measurement_requests.append(measurement_request)
        measurements_client = Measurements(wos_client)
        measurements_client.add(monitor_instance_id=fairness_monitor_instance_id, monitor_measurement_request=daily_measurement_requests)     
print('Finished')

Loading day 1


Loading day 2
Loading day 3


Loading day 4
Loading day 5


Loading day 6


Loading day 7


Finished


## Insert historical debias metrics

In [96]:
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_debias_v2.json
!ls -lh history_debias_v2.json

--2024-09-06 09:46:24--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_debias_v2.json


Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 742418 (725K) [text/plain]
Saving to: ‘history_debias_v2.json.2’

history_debias_v2.j 100%[===================>] 725.02K  --.-KB/s    in 0.02s   

2024-09-06 09:46:24 (38.8 MB/s) - ‘history_debias_v2.json.2’ saved [742418/742418]



-rw-rw----. 1 1000750000 wscommon 726K Sep  6 07:07 history_debias_v2.json


In [97]:
with open("history_debias_v2.json") as f:
    debias_values = json.load(f)
    for day in range(historyDays):
        print('Loading day', day + 1)
        daily_measurement_requests = []
        sources_list = []
        for hour in range(24):
            score_time = (datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
            index = (day * 24 + hour) % len(debias_values) # wrap around and reuse values if needed
            debias_values[index]["timestamp"] = score_time
            debias_value = debias_values[index]
            metrics_list = debias_value["metrics"]
            sources = debias_value["sources"]
            sources_list = []
            for source in sources:
                sources_list.append(
                    Source(
                        id=source["id"],
                        type=source["type"],
                        data=source["data"]
                    )
                )  
            measurement_request = MonitorMeasurementRequest(metrics=metrics_list, sources=sources_list, timestamp=score_time)
            daily_measurement_requests.append(measurement_request)
        measurements_client = Measurements(wos_client)
        measurements_client.add(monitor_instance_id=fairness_monitor_instance_id, monitor_measurement_request=daily_measurement_requests)         

print('Finished')

Loading day 1


Loading day 2


Loading day 3


Loading day 4


Loading day 5


Loading day 6


Loading day 7


Finished


## Insert historical quality metrics

In [98]:
measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    quality_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        
        metric = {"area_under_roc": measurements[day]}
                
        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [metric])
        quality_measurement_requests.append(measurement_request)
        
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=quality_monitor_instance_id,
                                            monitor_measurement_request=quality_measurement_requests).result    
    
print('Finished')

Loading day 1


Loading day 2


Loading day 3
Loading day 4


Loading day 5
Loading day 6


Loading day 7
Finished


## Insert historical confusion matrixes

In [99]:
!rm history_quality_metrics.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_quality_metrics.json
!ls -lh history_quality_metrics.json

--2024-09-06 09:46:28--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_quality_metrics.json


Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 80099 (78K) [text/plain]
Saving to: ‘history_quality_metrics.json’

history_quality_met 100%[===================>]  78.22K  --.-KB/s    in 0.005s  

2024-09-06 09:46:28 (14.0 MB/s) - ‘history_quality_metrics.json’ saved [80099/80099]



-rw-rw----. 1 1000750000 wscommon 79K Sep  6 09:46 history_quality_metrics.json


In [100]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Source

with open('history_quality_metrics.json') as json_file:
    records = json.load(json_file)
    
for day in range(historyDays):
    index = 0
    cm_measurement_requests = []
    print('Loading day', day + 1)
    
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"

        metric = records[index]['metrics']
        source = records[index]['sources']

        
        measurement_request = {"timestamp": score_time, "metrics": [metric], "sources": [source]}
        cm_measurement_requests.append(measurement_request)

        index+=1

    response = wos_client.monitor_instances.measurements.add(monitor_instance_id=quality_monitor_instance_id, monitor_measurement_request=cm_measurement_requests).result    

print('Finished')

Loading day 1


Loading day 2


Loading day 3


Loading day 4
Loading day 5


Loading day 6


Loading day 7
Finished


## Insert historical performance metrics

In [101]:
target = Target(
        target_type=TargetTypes.INSTANCE,
        target_id=payload_data_set_id
    )

parameters = dict()
performance_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=wos_client.monitor_definitions.MONITORS.PERFORMANCE.ID,
            target=target,
            parameters = parameters
).result
performance_monitor_instance_id = performance_monitor_instance_details.metadata.id




 Waiting for end of monitor instance creation e02e8e15-c68b-4337-8491-3b223bef02d2 






active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [102]:
for day in range(historyDays):
    performance_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        score_count = random.randint(60, 600)
        
        metric = {"record_count": score_count, "data_set_type": "scoring_payload"}
        
        measurement_request = {"timestamp": score_time, "metrics": [metric]}
        performance_measurement_requests.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=performance_monitor_instance_id,
                                            monitor_measurement_request=performance_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2


Loading day 3
Loading day 4
Loading day 5


Loading day 6
Loading day 7


Finished


## Insert historical drift measurements

In [103]:
!rm history_drift_measurement_*.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_0.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_1.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_3.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_4.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_5.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_6.json
!ls -lh history_drift_measurement_*.json

--2024-09-06 09:46:36--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_0.json


Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 

185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 850981 (831K) [text/plain]
Saving to: ‘history_drift_measurement_0.json’

          history_d   0%[                    ]       0  --.-KB/s               

history_drift_measu 100%[===================>] 831.04K  --.-KB/s    in 0.02s   

2024-09-06 09:46:37 (42.7 MB/s) - ‘history_drift_measurement_0.json’ saved [850981/850981]



...

--2024-09-06 09:46:40--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_6.json


Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 860041 (840K) [text/plain]
Saving to: ‘history_drift_measurement_6.json’

          history_d   0%[                    ]       0  --.-KB/s               

history_drift_measu 100%[===================>] 839.88K  --.-KB/s    in 0.02s   

2024-09-06 09:46:40 (41.8 MB/s) - ‘history_drift_measurement_6.json’ saved [860041/860041]



-rw-rw----. 1 1000750000 wscommon 832K Sep  6 09:46 history_drift_measurement_0.json
-rw-rw----. 1 1000750000 wscommon 868K Sep  6 09:46 history_drift_measurement_1.json
-rw-rw----. 1 1000750000 wscommon 870K Sep  6 09:46 history_drift_measurement_2.json
-rw-rw----. 1 1000750000 wscommon 910K Sep  6 09:46 history_drift_measurement_3.json
-rw-rw----. 1 1000750000 wscommon 841K Sep  6 09:46 history_drift_measurement_4.json
-rw-rw----. 1 1000750000 wscommon 836K Sep  6 09:46 history_drift_measurement_5.json
-rw-rw----. 1 1000750000 wscommon 840K Sep  6 09:46 history_drift_measurement_6.json


In [104]:
for day in range(historyDays):
    drift_measurements = []

    with open("history_drift_measurement_{}.json".format(day), 'r') as history_file:
        drift_daily_measurements = json.load(history_file)
    print('Loading day', day + 1)

    #Historical data contains 8 records per day - each represents 3 hour drift window.
    
    for nb_window, records in enumerate(drift_daily_measurements):
        for record in records:
            window_start =  datetime.utcnow() + timedelta(hours=(-(24 * day + (nb_window+1)*3 + 1))) # first_payload_record_timestamp_in_window (oldest)
            window_end = datetime.utcnow() + timedelta(hours=(-(24 * day + nb_window*3 + 1)))# last_payload_record_timestamp_in_window (most recent)
            #modify start and end time for each record
            record['sources'][0]['data']['start'] = window_start.isoformat() + "Z"
            record['sources'][0]['data']['end'] = window_end.isoformat() + "Z"
            
            
            metric = record['metrics'][0]
            source = record['sources'][0]

            measurement_request = {"timestamp": window_start.isoformat() + "Z", "metrics": [metric], "sources": [source]}
            
            drift_measurements.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=drift_monitor_instance_id,
                                            monitor_measurement_request=drift_measurements).result    

    
    print("Daily loading finished.")

Loading day 1


Daily loading finished.
Loading day 2


Daily loading finished.
Loading day 3


Daily loading finished.
Loading day 4
Daily loading finished.
Loading day 5


Daily loading finished.
Loading day 6


Daily loading finished.
Loading day 7


Daily loading finished.


## Additional data to help debugging

In [105]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', ai_func_deployment_uid)

Datamart: 00000000-0000-0000-0000-000000000000
Model: 4a60bd7f-9640-411f-8791-7cb4c56ff918
Deployment: b51ed35d-58a5-4def-8b18-ca3d83185a38


## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [106]:
wos_client.data_sets.show_records(payload_data_set_id, limit=5)

None,4652,c9876b10-f190-475e-b570-829e7266231d-1,0.0,4.0,0.0,2,unknown,2024-09-06T09:37:37.509Z,0.5465858822615995,"[20, [2, 3, 7, 11, 13, 14, 15, 16, 17, 18, 19], [4.0, 4.0, 1.0, 1.0, 33.0, 4652.0, 2.0, 2.0, 40.0, 2.0, 1.0]]",no_checking,appliances,0.0,No Risk,1.0,0.0,skilled,none,4.0,0.0,none,33,40,prior_payments_delayed,0.0,2,1.0,0.0,0.0,0.0,yes,own,0.0,2,b51ed35d-58a5-4def-8b18-ca3d83185a38,"[10.931717645231991, 9.068282354768009]",yes,0.0,male,1,car_other,1_to_4,"[0.5465858822615995, 0.45341411773840046]"
None,250,c9876b10-f190-475e-b570-829e7266231d-10,0.0,0.0,2.0,1,less_100,2024-09-06T09:37:37.509Z,0.9149100675246604,"[2.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 2.0, 2.0, 0.0, 3.0, 0.0, 0.0, 5.0, 250.0, 1.0, 2.0, 33.0, 1.0, 1.0]",0_to_200,car_new,0.0,No Risk,2.0,0.0,unemployed,bank,0.0,3.0,none,5,33,all_credits_paid_back,2.0,2,0.0,3.0,3.0,0.0,yes,own,0.0,1,b51ed35d-58a5-4def-8b18-ca3d83185a38,"[18.298201350493205, 1.7017986495067932]",none,0.0,male,1,real_estate,less_1,"[0.9149100675246604, 0.08508993247533968]"
None,250,c9876b10-f190-475e-b570-829e7266231d-100,0.0,0.0,1.0,1,less_100,2024-09-06T09:37:37.509Z,0.9176227997101503,"[20, [0, 1, 4, 8, 9, 13, 14, 15, 16, 17, 18, 19], [1.0, 3.0, 4.0, 1.0, 1.0, 10.0, 250.0, 1.0, 1.0, 28.0, 1.0, 1.0]]",less_0,car_new,0.0,No Risk,0.0,0.0,skilled,stores,0.0,3.0,none,10,28,all_credits_paid_back,1.0,1,0.0,4.0,0.0,0.0,yes,rent,1.0,1,b51ed35d-58a5-4def-8b18-ca3d83185a38,"[18.352455994203005, 1.6475440057969937]",none,0.0,male,1,savings_insurance,unemployed,"[0.9176227997101503, 0.08237720028984968]"
None,9094,c9876b10-f190-475e-b570-829e7266231d-101,0.0,10.0,0.0,4,greater_1000,2024-09-06T09:37:37.509Z,0.619015287312167,"[20, [2, 3, 4, 7, 13, 14, 15, 16, 17, 18, 19], [10.0, 3.0, 2.0, 3.0, 31.0, 9094.0, 4.0, 3.0, 44.0, 2.0, 1.0]]",no_checking,other,0.0,Risk,3.0,1.0,skilled,none,3.0,0.0,none,31,44,prior_payments_delayed,0.0,3,0.0,2.0,0.0,0.0,yes,own,0.0,2,b51ed35d-58a5-4def-8b18-ca3d83185a38,"[7.61969425375666, 12.38030574624334]",none,0.0,male,1,unknown,greater_7,"[0.380984712687833, 0.619015287312167]"
None,2074,c9876b10-f190-475e-b570-829e7266231d-102,0.0,0.0,0.0,4,100_to_500,2024-09-06T09:37:37.509Z,0.5159135280965452,"[20, [3, 4, 7, 11, 13, 14, 15, 16, 17, 18, 19], [1.0, 2.0, 1.0, 1.0, 8.0, 2074.0, 4.0, 3.0, 37.0, 2.0, 1.0]]",no_checking,car_new,0.0,Risk,1.0,1.0,skilled,none,1.0,0.0,none,8,37,prior_payments_delayed,0.0,3,1.0,2.0,0.0,0.0,yes,own,0.0,2,b51ed35d-58a5-4def-8b18-ca3d83185a38,"[9.681729438069096, 10.318270561930904]",yes,0.0,male,1,car_other,greater_7,"[0.48408647190345483, 0.5159135280965452]"


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the OpenScale Dashboard: (https://url-to-your-cp4d-cluster/aiopenscale). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.
